<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/button_radar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
import pickle
import numpy as np
import keras
from keras.models import Sequential, load_model
from keras.layers import Flatten, Dense


def select_random_subarrays(X, Y, num_samples=20):
    if len(X) != len(Y):
        raise ValueError("The X and Y dimensions must match")
    X_selected = []
    Y_selected = []
    selected_count = 0
    selected_indices = []
    while selected_count < num_samples:
        random_index = np.random.randint(0, len(X))
        if random_index not in selected_indices:
            X_selected.append(X[random_index])
            Y_selected.append(Y[random_index])
            selected_indices.append(random_index)
            selected_count += 1
    return X_selected, Y_selected

def normalize_array_2d(array, max_value=800, new_max_value=255):
    if not isinstance(array, np.ndarray):
        raise TypeError("Input array must be a NumPy array.")
    if array.ndim != 2:
        raise ValueError("Input array must be a two-dimensional array.")
    if np.min(array) < 0:
        raise ValueError("Input array cannot contain negative values.")
    normalized_array = array / max_value
    rescaled_array = normalized_array * new_max_value
    return rescaled_array

def convert_to_bit_array(double_array):
    bit_array = []
    for sublist in double_array:
        binary_sublist = [int(bit) for num in sublist for bit in format(int(num), '08b')]
        bit_array.append(binary_sublist)
    return bit_array

In [159]:
with open('radar_array_train.txt', 'rb') as radar:
  x_train = pickle.load(radar)

with open('button_array_train.txt', 'rb') as button:
  y_train = pickle.load(button)

x_selected, y_selected = select_random_subarrays(x_train, y_train)
x_train = np.array(x_selected)
y_train = np.array(y_selected)
x_train_normalized = normalize_array_2d(x_train, max_value=800, new_max_value=255)
x_train_bit_array = convert_to_bit_array(x_train_normalized)
x_train_bit_array = np.array(x_train_bit_array)
print(x_train)
print(x_train_normalized)
print(x_train_bit_array)
print(y_train)
print(len(x_train))
print(len(y_train))

[[125. 275. 190. 410.]
 [100.  80. 385. 215.]
 [125. 275. 185. 415.]
 [125.  95. 125. 475.]
 [160.  80. 485. 115.]
 [295. 265. 310. 160.]
 [240. 160.  40. 400.]
 [160.  80. 485. 115.]
 [410. 150. 100. 290.]
 [125.  95. 125. 475.]
 [205. 195. 200.  50.]
 [360. 200. 310.  40.]
 [160. 400. 310. 290.]
 [220. 180. 200.  50.]
 [160.  80. 485. 115.]
 [ 75. 485. 280. 150.]
 [160.  80. 485. 115.]
 [160.  80. 485. 115.]
 [480.  80. 310. 290.]
 [160.  80. 485. 115.]]
[[ 39.84375  87.65625  60.5625  130.6875 ]
 [ 31.875    25.5     122.71875  68.53125]
 [ 39.84375  87.65625  58.96875 132.28125]
 [ 39.84375  30.28125  39.84375 151.40625]
 [ 51.       25.5     154.59375  36.65625]
 [ 94.03125  84.46875  98.8125   51.     ]
 [ 76.5      51.       12.75    127.5    ]
 [ 51.       25.5     154.59375  36.65625]
 [130.6875   47.8125   31.875    92.4375 ]
 [ 39.84375  30.28125  39.84375 151.40625]
 [ 65.34375  62.15625  63.75     15.9375 ]
 [114.75     63.75     98.8125   12.75   ]
 [ 51.      127.5      

In [160]:
X = x_train_bit_array
y = y_train

start = y.shape[1]
stop = X.shape[1]

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def func1(num_hidden_layer, epochs, X, y):
    input_neurons = X.shape[1]
    hidden_neurons = num_hidden_layer
    output_neurons = y.shape[1]
    weights_input_hidden = np.random.rand(input_neurons, hidden_neurons)
    weights_hidden_output = np.random.rand(hidden_neurons, output_neurons)
    for i in range(epochs):
        hidden_layer_input = np.dot(X, weights_input_hidden)
        hidden_layer_output = sigmoid(hidden_layer_input)
        output_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
        output = sigmoid(output_layer_input)
        error = y - output
        output_gradient = output * (1 - output) * error
        error_hidden = output_gradient.dot(weights_hidden_output.T)
        hidden_gradient = hidden_layer_output * (1 - hidden_layer_output) * error_hidden
        weights_hidden_output += hidden_layer_output.T.dot(output_gradient)
        weights_input_hidden += X.T.dot(hidden_gradient)
    return weights_input_hidden, weights_hidden_output


def test1(weights_input_hidden, weights_hidden_output, X):
    hidden_layer_input = np.dot(X, weights_input_hidden)
    hidden_layer_output = sigmoid(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
    output = sigmoid(output_layer_input)
    return output


def find_best_coefficients1(X, y):
    best_error = np.inf
    global best_num_hidden_layer, best_epoch
    for num_hidden_layer in range(start+(stop//3), stop//2):
        for epoch in range(2000, 2001):
            weights_input_hidden, weights_hidden_output = func1(num_hidden_layer, epoch, X, y)
            training_error = np.sum(np.square(y - test1(weights_input_hidden, weights_hidden_output, X)))
            if training_error < best_error:
                best_error = training_error
                best_num_hidden_layer, best_epoch  =  num_hidden_layer, epoch
    return best_num_hidden_layer, best_epoch


num_hidden_layer, epochs = find_best_coefficients1(X, y)
print("Best num_hidden_layer:", num_hidden_layer)
print("Best epoch:", epochs)

weights_input_hidden, weights_hidden_output = func1(num_hidden_layer, epochs, X, y)

# print(weights_input_hidden)
# print(weights_hidden_output)

test_error = np.sum(np.square(y - test1(weights_input_hidden, weights_hidden_output, X)))
print("Test error:", "{:.5f}".format(test_error))

print("Predictions on test data:")
print(test1(weights_input_hidden, weights_hidden_output, np.array([X[0]])))

Best num_hidden_layer: 14
Best epoch: 2000
Test error: 10.00002
Predictions on test data:
[[1.27427200e-04 9.63581239e-09 9.46806418e-04 1.07769450e-08]]


In [161]:
def func2(A, B, C, D, E, X, y):
    input_neurons = X.shape[1]
    hidden_neurons = num_hidden_layer
    output_neurons = y.shape[1]
    weights_input_hidden = A * np.random.rand(input_neurons, hidden_neurons) + B
    weights_hidden_output = C * np.random.rand(hidden_neurons, output_neurons) + D
    for i in range(epochs):
        hidden_layer_input = np.dot(X, weights_input_hidden)
        hidden_layer_output = sigmoid(hidden_layer_input)
        output_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
        output = sigmoid(output_layer_input)
        error = y - output
        output_gradient = output * (1 - output) * error
        error_hidden = output_gradient.dot(weights_hidden_output.T)
        hidden_gradient = hidden_layer_output * (1 - hidden_layer_output) * error_hidden
        weights_hidden_output += E * hidden_layer_output.T.dot(output_gradient)
        weights_input_hidden += E * X.T.dot(hidden_gradient)
    return weights_input_hidden, weights_hidden_output


def test2(weights_input_hidden, weights_hidden_output, X):
    hidden_layer_input = np.dot(X, weights_input_hidden)
    hidden_layer_output = sigmoid(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
    output = sigmoid(output_layer_input)
    return output


def find_best_coefficients2(X, y):
    best_error = np.inf
    global best_A, best_B, best_C, best_D, best_E
    for A in range(-1, 1):
        for B in range(-1, 1):
            for C in range(-1, 1):
                for D in range(-1, 1):
                    for E in range(-1, 2):
                        weights_input_hidden, weights_hidden_output = func2(A, B, C, D, E, X, y)
                        training_error = np.sum(np.square(y - test2(weights_input_hidden, weights_hidden_output, X)))
                        if training_error < best_error:
                            best_error = training_error
                            best_A, best_B, best_C, best_D, best_E = A, B, C, D, E
    return best_A, best_B, best_C, best_D, best_E


A, B, C, D, E = find_best_coefficients2(X, y)
print("Best coefficients:", A, B, C, D, E)

weights_input_hidden, weights_hidden_output = func2(A, B, C, D, E, X, y)

# print(weights_input_hidden)
# print(weights_hidden_output)

test_error = np.sum(np.square(y - test2(weights_input_hidden, weights_hidden_output, X)))
print("Test error:", "{:.5f}".format(test_error))

print("Predictions on test data:")
print(test1(weights_input_hidden, weights_hidden_output, np.array([X[0]])))

Best coefficients: 0 0 -1 0 1
Test error: 0.00071
Predictions on test data:
[[0.00059493 0.00554038 0.00061302 0.00722482]]
